### **MUHAMMAD SHOFA DZIKRON_HOMEWORK**

In [1]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 37.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=6bf10ca81e342f03d3fe279efa1dbcde05995bbee39626b769ca5eabbd78fe5b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [53]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("minAmount").getOrCreate()

In [54]:
spark

In [55]:
schema = StructType([ \
                     StructField("InvoiceNo", IntegerType(), True), \
                     StructField("StockCode", StringType(), True), \
                     StructField("Description", StringType(), True), \
                     StructField("Quantity", IntegerType(), True), \
                     StructField("InvoiceData", IntegerType(), True), \
                     StructField("Amount", FloatType(), True), \
                     StructField("CustomerID", FloatType(), True), \
                     StructField("Country", StringType(), True)])

In [56]:
df = spark.read.schema(schema).csv("retail-data-full.csv")
df.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceData: integer (nullable = true)
 |-- Amount: float (nullable = true)
 |-- CustomerID: float (nullable = true)
 |-- Country: string (nullable = true)



In [57]:
MinAmount = df.select("CustomerID","Amount")

MinAmount.show()

+----------+------+
|CustomerID|Amount|
+----------+------+
|   17850.0|  2.55|
|   17850.0|  3.39|
|   17850.0|  2.75|
|   17850.0|  3.39|
|   17850.0|  3.39|
|   17850.0|  7.65|
|   17850.0|  4.25|
|   17850.0|  1.85|
|   17850.0|  1.85|
|   13047.0|  1.69|
|   13047.0|   2.1|
|   13047.0|   2.1|
|   13047.0|  3.75|
|   13047.0|  1.65|
|   13047.0|  4.25|
|   13047.0|  4.95|
|   13047.0|  9.95|
|   13047.0|  5.95|
|   13047.0|  5.95|
|   13047.0|  7.95|
+----------+------+
only showing top 20 rows



In [66]:
minAmount = MinAmount.groupBy("CustomerID").min("Amount").sort("CustomerID")
minAmount.show()

+----------+-----------+
|CustomerID|min(Amount)|
+----------+-----------+
|      null|        0.0|
|   12431.0|       0.85|
|   12433.0|       0.29|
|   12472.0|       1.25|
|   12583.0|       0.42|
|   12662.0|       0.42|
|   12748.0|       4.95|
|   12791.0|       1.85|
|   12838.0|       0.29|
|   12868.0|       0.65|
|   12921.0|       1.25|
|   12947.0|       0.42|
|   13047.0|       1.65|
|   13065.0|       0.42|
|   13093.0|       0.85|
|   13255.0|       1.25|
|   13408.0|       1.25|
|   13448.0|       0.29|
|   13468.0|       1.25|
|   13576.0|       1.25|
+----------+-----------+
only showing top 20 rows



In [67]:
hasil = minAmount.withColumn("Amount",
                          func.col("min(Amount)"))\
                          .select("CustomerID", "Amount").sort("CustomerID")

In [68]:
results = hasil.collect()

for result in results:
    print(result[0], "\t{:.2f}".format(result[1]))

None 	0.00
12431.0 	0.85
12433.0 	0.29
12472.0 	1.25
12583.0 	0.42
12662.0 	0.42
12748.0 	4.95
12791.0 	1.85
12838.0 	0.29
12868.0 	0.65
12921.0 	1.25
12947.0 	0.42
13047.0 	1.65
13065.0 	0.42
13093.0 	0.85
13255.0 	1.25
13408.0 	1.25
13448.0 	0.29
13468.0 	1.25
13576.0 	1.25
13694.0 	0.24
13705.0 	0.65
13747.0 	9.95
13748.0 	2.55
13758.0 	1.25
13767.0 	0.55
13777.0 	1.05
14001.0 	1.69
14045.0 	2.55
14078.0 	0.42
14142.0 	0.21
14237.0 	0.85
14307.0 	0.21
14496.0 	1.25
14527.0 	27.50
14594.0 	0.42
14606.0 	0.29
14688.0 	0.38
14696.0 	0.42
14729.0 	0.29
14849.0 	0.55
14911.0 	0.65
15012.0 	0.42
15100.0 	10.95
15165.0 	1.45
15235.0 	0.95
15291.0 	2.55
15311.0 	0.85
15350.0 	2.10
15485.0 	0.42
15513.0 	2.55
15525.0 	0.29
15605.0 	0.85
15862.0 	0.55
15922.0 	1.25
15983.0 	0.42
16029.0 	1.25
16048.0 	0.42
16098.0 	0.85
16210.0 	0.72
16218.0 	0.55
16250.0 	0.42
16274.0 	0.29
16456.0 	1.65
16539.0 	0.85
16552.0 	0.85
16583.0 	1.25
16835.0 	2.10
16928.0 	1.25
16955.0 	0.85
17025.0 	0.65
17069.0

In [51]:
spark.stop()